<a href="https://colab.research.google.com/github/quantranvr/all-in-one/blob/main/QA_with_RAG_series_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding logic for incorporating historical messages

To do so, we need to **update** our **prompt** to support historical messages as an input and add a sub-chain that takes the latest user question and **reformulates** it in a **context** from past messages

Tutorial @ https://python.langchain.com/docs/use_cases/question_answering/chat_history

This notebook contains 2 core parts:
1. **Reproduce** [tutorial](https://python.langchain.com/docs/use_cases/question_answering/chat_history)'s example
2. **Apply** knowledge learned to similar problem

# Installation

In [1]:
!pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community chromadb bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Part 1: Reproduce

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [13]:
# load
import bs4
from langchain_community.document_loaders import WebBaseLoader
# split
from langchain.text_splitter import RecursiveCharacterTextSplitter
# index
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
# retrieve & generate
from langchain import hub
from langchain_openai import ChatOpenAI
# chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# contextualize question
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

In [4]:
# load documents
web_paths = (
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
)

bs4_strainer = bs4.SoupStrainer(
    class_=("post-content", "post-title", "post-header")
)

loader = WebBaseLoader(
    web_paths = web_paths,
    bs_kwargs = {"parse_only": bs4_strainer}
)

docs = loader.load()

print(f"Loaded document has {len(docs[0].page_content)} characters")

Loaded document has 42824 characters


In [5]:
# split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

splits = splitter.split_documents(docs)

print(f"Number of chunks = {len(splits)}")

Number of chunks = 66


In [6]:
# store and index chunks
vectorstore = Chroma.from_documents(
    documents = splits,
    embedding = OpenAIEmbeddings(),
)

In [7]:
# retrieve & generate
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
# answer question
rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done through various methods such as using prompting techniques, task-specific instructions, or human inputs. The goal is to make the task more manageable and facilitate the interpretation of the model's thinking process."

In [17]:
# contextualize the question
contextualize_q_system_prompt = """\
Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is.\
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

contextualize_q_chain = (
    contextualize_q_prompt
    | llm
    | StrOutputParser()
)

In [19]:
chat_history = [
    HumanMessage(content="What does LLM stand for?"),
    AIMessage(content="Large language model"),

]

In [21]:
# chat prompt
contextualize_q_prompt.invoke(
    {
        "chat_history": chat_history,
        "question": "What is meant by large"
    }
)

ChatPromptValue(messages=[SystemMessage(content='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.'), HumanMessage(content='What does LLM stand for?'), AIMessage(content='Large language model'), HumanMessage(content='What is meant by large')])

In [20]:
# answer
contextualize_q_chain.invoke(
    {
        "chat_history": chat_history,
        "question": "What is meant by large",
    }
)

'What is the definition of "large" in the context of a language model?'

In [22]:
# chain with chat history
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [25]:
# answer and add to history
chat_history = []

question = "What is Task Decomposition?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What are common ways of doing it?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='Common ways of task decomposition include:\n1. Using techniques like Chain of Thought (CoT) or Tree of Thoughts, where the task is broken down into multiple manageable steps, allowing the model to think step by step and explore different reasoning possibilities.\n2. Providing task-specific instructions or prompts to guide the model in decomposing the task. For example, asking the model to outline a story for writing a novel or asking for subgoals to achieve a specific task.\n3. Involving human inputs, where humans provide guidance or input to help decompose the task into smaller steps. This can be done through collaboration or by leveraging human expertise in the task domain.')